# Laplacian Convolutional Representation (LCR)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox(z, w, lmbda, denominator):
    T = z.shape[0]
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    ell_t = laplacian(T, tau)
    denominator = lmbda + gamma * np.fft.fft2(np.outer(ell_s, ell_t)) ** 2
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = prox(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

In [2]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.045706426428106114
3.5989219479929093

20
0.02393589280354179
2.2593377894748596

30
0.02260321135539069
2.1549756215799984

40
0.02189096983410263
2.0955480434054663

50
0.021428145373003516
2.0566811247350665

60
0.021097781101543426
2.0291879627538347

70
0.020850163362595136
2.008841143121188

80
0.020657582733725473
1.993253440828172

90
0.020505604789758842
1.9810895440152554

100
0.020383741159491684
1.971456948710559

Running time: 1796 seconds.


In [3]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.5
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.22684152932986826
15.917221724709215

20
0.027222786730076345
2.4440000870305973

30
0.023940061848754295
2.273446872416439

40
0.023140483525337822
2.2135226875696206

50
0.02265398559933648
2.175141727525689

60
0.02231517291374989
2.148132010566657

70
0.022064467857462897
2.128177621788459

80
0.021871139740507833
2.112941430030623

90
0.021719167711484647
2.101043407870563

100
0.021597700181477573
2.09161653619306

Running time: 1726 seconds.


In [4]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.7
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.7069716046327081
53.57415031327299

20
0.08706284416435582
6.251867728241243

30
0.029409260570430123
2.59752953665467

40
0.025857405767885144
2.443254365053087

50
0.025119747686206267
2.4039389971513425

60
0.024732428587175035
2.379285136043403

70
0.024468549680783803
2.361578667454719

80
0.024273870712478107
2.3481860760806263

90
0.02412400077130145
2.337762506697902

100
0.024005603630901005
2.3295130668122046

Running time: 1715 seconds.


In [5]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-9 * N * T
gamma = 5 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

10
0.8462112461592463
61.59175972313382

20
0.4015949323255642
30.4638292470016

30
0.1338626009465662
10.354673039519097

40
0.053886739887361985
4.057618351028555

50
0.04065519616352157
3.3707003147463204

60
0.03652421264474001
3.2157434500779427

70
0.03458745182642591
3.1456587546277626

80
0.0339037998778549
3.127574486077418

90
0.033612677259052884
3.121011401181271

100
0.033455638911586295
3.117503396379154

Running time: 1684 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>